In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Created on Thu Jul  4 12:02:07 2019

This programm should be a cleaned up version of r300-ic-korrelationstest.py
and should be able to correct the JJ data for inherent fabrication errors of
the JJ area and the critical current suppression of the critical current of
the JJ.
"""


import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from datetime import date
import pandas as pd
from scipy.optimize import curve_fit
#from statistics import mean


__author__ = 'Felix Herrmann'
__version__ = '0.3'
__status__ = 'Prototyp'


"""
TODO:
- implement automated data collection
    data input needs to be impruved (2019-07-15)
- test (2019-07-05 done for CorrectJJData)
- how do I delet singel datapoints? (use filter??)
"""


# Test data
r300jj = np.array([220,136,89,183,310,243,93,264,218,89,84,265,216,87,79,269,221])
i0jjpos = np.array([2.63,7.42,16.4,3.62,1.85,2.4,15.9,2.00,2.89,17.9,19.9,1.89,2.77,17.2,20,0.68,2.28])
i0jjneg = np.array([2.62,8.09,17.5,3.5,2.15,2.67,16.6,2.28,2.95,18.3,19.5,2.1,2.83,17.5,20.4,0.7,2.57])
jjdiameter = np.array([4.5,6,8,5,4,4.5,8,4,4.5,8,8.5,4,4.5,8,8.5,4,4.5])


"""
Class to import data from storage. # needs to be improved (2019-07-15)
"""
# automated input of multiple files implemented in main program
# this needs to be moved here (2019-09-30)

class DataInput:
    
    def __init__(self,path,file):
        """Get data from $PATH and store it in a meaningfull way."""
        self.data = pd.read_table(str(path)+'/'+str(file),delim_whitespace=True,
                                  comment='/', skiprows=2,
                                  names=['Size','R300','Ic,pos','Ic,neg',
                                  'Rn,pos','Rn,neg','Vg,pos','Vg,neg',
                                  'Ir,pos','Ir,neg','ICPoint,pos','ICPoint,neg',
                                  'Rs2mV,pos','Rs2mV,neg','Proxy1,pos',
                                  'Proxy2,pos','Proxy1,neg','Proxy2,neg','Name'])

    """Remove NaN from datasets and return sperated arrays."""
    def size(self):
        x = (pd.Series(self.data['Size']).values)
        x = x[np.logical_not(np.isnan(x))]
        y = np.array([])
        for i in range(len(x)):
            y = np.append(y,x[i])
        return y

    def r300(self):
        x = (pd.Series(self.data['R300']).values)
        x = x[np.logical_not(np.isnan(x))]
        y = np.array([])
        for i in range(len(x)):
            y = np.append(y,x[i])
        return y

    def i0pos(self):
        x = (pd.Series(self.data['Ic,pos']).values)
        x = x[np.logical_not(np.isnan(x))]
        y = np.array([])
        for i in range(len(x)):
            y = np.append(y,x[i])
        return y

    def i0neg(self):
        x = (pd.Series(self.data['Ic,neg']).values)
        x = x[np.logical_not(np.isnan(x))]
        y = np.array([])
        for i in range(len(x)):
            y = np.append(y,x[i])
        return y

    def rnpos(self):
        x = (pd.Series(self.data['Rn,pos']).values)
        x = x[np.logical_not(np.isnan(x))]
        y = np.array([])
        for i in range(len(x)):
            y = np.append(y,x[i])
        return y

    def rnneg(self):
        x = (pd.Series(self.data['Rn,neg']).values)
        x = x[np.logical_not(np.isnan(x))]
        y = np.array([])
        for i in range(len(x)):
            y = np.append(y,x[i])
        return y


"""
Class to correct for fabrication errors of the area and the critical current
suppression of the critical current of the JJ.
"""
class CorrectJJData:

    def __init__(self,r300_jj,i0_jj_pos,i0_jj_neg,jj_diameter,rn_pos,rn_neg):
        self.r300 = r300_jj     # roomtemperatur resistance of the JJ
        """
        i0_jj_pos and i0_jj_neg are measured to avoid offsets in the
        measurement electronic.
        They are averaged to get to (uncorrected) critical current of the JJ.
        """
        # uncorrected critical current of the JJ
        self.i0_measured = (abs(i0_jj_pos) + abs(i0_jj_neg)) / 2
        self.diameter_design = jj_diameter  # design diamenter of JJ
        self.area_design = jj_diameter**2   # design area of JJ
        # resistance in the superconducting state
        self.rn = (rn_pos + rn_neg) / 2
        self.area_corrected = self.area_design
    
    def func(self,x,m,b):
        return m*x+b
        
    def jj_area_correction_optimization_function(self,correction):
        area = (self.diameter_design + correction)**2
        x = 1/area
        popt, pcov = curve_fit(self.func,x,self.rn)
        self.rho_n=popt[0]
        return popt[1]
        
    
    def jj_area_correction(self):
        """
        Fit r300 over 1/area.
        The plot should intersect with origin.
        Calcualte the offset on the x-axis, this corresponds to the
        fabrication error of the JJ area.
        """
     #   try:
        res = optimize.leastsq(self.jj_area_correction_optimization_function,0)
        self.diameter_correction = res[0]
        self.diameter_corrected = self.diameter_design + self.diameter_correction
        self.area_corrected = self.diameter_corrected**2
       # except:
       #     # debug artefact, can this be removed? (2019-07-18)
       #     x = np.array([])
       #     for i in np.linspace(-10,10,1000):
       #         x = np.append(x,self.jj_area_correction_optimization_function(i))
       #     plt.plot(np.linspace(-10,10,1000),x)
   

    def plot_jj_area_resistance(self,plot_design,save_plot):
        """
        Plots the roomtemperature resistance of the JJ over the corrected JJ area.
        plot_design == True plots additional with the design JJ area.
        save_plot == True will save the plot in the current directory.
        """
        fig, ax = plt.subplots()
        if plot_design == True:
            plt.plot(1/self.area_design,self.r300,'*',label='design area')
            fit_design_area = np.polyfit(1/self.area_design,self.r300,1)
            plt.plot(1/self.area_design,fit_design_area[0]*1/self.area_design
                        + fit_design_area[1],label='fit design area')
        plt.plot(1/self.area_corrected,self.r300,'.',label='corrected area')
        fit_corrected_area = np.polyfit(1/self.area_corrected,self.r300,1)
        plt.plot(1/self.area_corrected,fit_corrected_area[0]*1/self.area_corrected
                    + fit_corrected_area[1],label='fit corrected area')
        ax.annotate('$\Delta W$ = '+str(np.round(float(self.diameter_correction),2))+' $\mu m$'
                    +'\n $rho_N$ = '+str(np.round(fit_corrected_area[0],2))+' $\Omega \mu m^2$',
                    xy=(.9,.25),xycoords='figure fraction',
                    horizontalalignment='right',verticalalignment='bottom')
        plt.ylabel('$R300$ [$\Omega$]')
        plt.xlabel('$1/A_{JJ}$ [$1/\mu m^2$]')
        plt.legend()
        if save_plot == True:
            today = str(date.today())
            plt.savefig(today+'-jj-inverse-area-r300.png')
        plt.show()
        print('Plot of the roomtemperature resistance over the area of the JJ.')

    def plot_jj_area_rn(self,plot_design,save_plot):
        """
        Plots the roomtemperature resistance of the JJ over the corrected JJ area.
        plot_design == True plots additional with the design JJ area.
        save_plot == True will save the plot in the current directory.
        """
        fig, ax = plt.subplots()
        if plot_design == True:
            plt.plot(1/self.area_design,self.rn,'*',label='design area')
            fit_design_area = np.polyfit(1/self.area_design,self.rn,1)
            plt.plot(1/self.area_design,fit_design_area[0]*1/self.area_design
                        + fit_design_area[1],label='fit design area')
        plt.plot(1/self.area_corrected,self.rn,'.',label='corrected area')
        fit_corrected_area = np.polyfit(1/self.area_corrected,self.rn,1)
        plt.plot(1/self.area_corrected,fit_corrected_area[0]*1/self.area_corrected
                    + fit_corrected_area[1],label='fit corrected area')
        ax.annotate('$\Delta W$ = '+str(np.round(float(self.diameter_correction),2))+' $\mu m$'
                    +'\n $rho_N$ = '+str((np.round(fit_corrected_area[0],2)))+' $\Omega \mu m^2$',
                    xy=(.9,.25),xycoords='figure fraction',
                    horizontalalignment='right',verticalalignment='bottom')
        plt.ylabel('$R_N$ [$\Omega$]')
        plt.xlabel('$1/A_{JJ}$ [$1/\mu m^2$]')
        plt.legend()
        if save_plot == True:
            today = str(date.today())
            plt.savefig(today+'-jj-inverse-area-rn.png')
        plt.show()
        print('Plot of the low temperature resistance over the area of the JJ.')

    def i0corr(self,i0,area,kappa):
        """
        Correction function for the noise suppression of the cricical
        current according to Kempf et al., Characterization of the reliability
        and uniformity of an anodization-free fabrication process for
        high-quality Nb/Al–AlO x /Nb Josephson junctions (2016), section 4.
        """
        return i0/(1-kappa*area**(-2/3))
       
    def min_b(self,kappa,jjareacorrected,i0jj):
        """
        Function for optimizing $\kappa$ so that $I_0(A_ {JJ, corr})$
        intersects with origen.
        """
        polyfit_i0corr = np.polyfit(jjareacorrected,
                                    self.i0corr(i0jj,jjareacorrected,kappa),1)
        return polyfit_i0corr[1] 

    def crit_current_suppression_correction(self,startvalue = 0):
        """
        The critical current of the JJ is suppressed by thermal noise.
        It holds that $ i0_measured / i0_corrected = ( 1 - kappa * A^(-2/3) ) $
        with parameter $kappa$ depending on crit current density, plasma
        frequency, temperature and the measurement current sweep rate.
        The crit current over the (corrected) JJ area should intersect with
        the origin, therefore $\kappa$ can be optimized to fullfill that.
        """        
        """ Startvalue for optimization is determined by human observer."""        
        try:
            res1 = optimize.leastsq(self.min_b,startvalue,
                                    args=(self.area_corrected, self.i0_measured))
            self.kappa = res1[0]
        except:
            print('Given startvalue is bad.')
            kappa = np.linspace(-50,50,1000)
            data = np.array([])
            for i in range(1000):
                data = np.append(data,self.min_b(kappa[i],self.area_corrected,
                                                 self.i0_measured))
            plt.plot(kappa,data,'.')
            plt.plot(kappa,np.zeros(len(kappa)))
            plt.xlabel('$\kappa$ [$\mu m^{2/3}$]')
            plt.ylabel('$I_{0}-offset$ [$\mu A$]')
            plt.ylim(-10,10)
            plt.show()
            startvalue = input('Please input x-value of the root point from the plot:\n')
            startvalue = int(startvalue)
            res2 = optimize.leastsq(self.min_b,startvalue,
                                    args=(self.area_corrected, self.i0_measured))
            self.kappa = res2[0]
        self.i0_corrected = self.i0corr(self.i0_measured,self.area_corrected,
                                    self.kappa)

    def plot_jj_corr_area_i0(self,save_plot):
        """
        Plots the critical current over the corrected JJ area.
        save_plot == True will save the plot in the current directory.
        """
        fig, ax = plt.subplots()
        plt.plot(self.area_corrected,self.i0_corrected,'.',label=('korrigiert'))
        plt.plot(self.area_corrected,self.i0_measured,'.',label=('gemessen'))
        fit = np.polyfit(self.area_corrected,self.i0_corrected,1)
        self.j_c_corr=fit[0]
        xplot=np.linspace(0,max(self.area_corrected),1000)
        plt.plot(xplot,fit[0]*xplot+fit[1])
        ax.annotate('$\kappa$ = '+str(np.round(float(self.kappa),2))+' $\mu m^{2/3}$'
                    +'\n $j_c$= '+str(np.round(fit[0],2))+' $\mu A / \mu m^2$',
                    xy=(.9,.25),xycoords='figure fraction',
                    horizontalalignment='right',verticalalignment='bottom')
        plt.xlabel('$A_{JJ, corr}$ [$\mu m^2$]')
        plt.ylabel('$I_{0, corr}$ [$\mu A$]')
        plt.legend()
        if save_plot == True:
            today = str(date.today())
            plt.savefig(today+'-jj-corr-area-corr-i0.png')
        plt.show()
        print('Plot of the corrected critical current over the corrected JJ area.')

    def calc_jj_crit_current_density(self):
        """The critical current is tranferd into the critical current density."""
        self.crit_current_density = self.i0_corrected/self.area_corrected

    def plot_jj_crit_current_density(self,save_plot):
        """
        Plots the critical current density over the inverse roomtemperature
        resistance devided by the correceted area of the JJ.
        save_plot == True will save the plot in the current directory.
        """
        fig, ax = plt.subplots()
        plt.plot(1/(self.r300*self.area_corrected),self.crit_current_density,'.')
        plt.xlabel('1/rho(300K) [$\Omega^{-1} \mu m^{-2}$]')
        plt.ylabel('$J_{c}$ [$\mu A / \mu m^{2}$]')
        if save_plot == True:
            today = str(date.today())
            plt.savefig(today+'-jj-inverse-rho300-crit-current-density.png')
        plt.show()
        print('Plot of the critical current density over the inverse roomtemperature resistance')
        print('devided by the corrected JJ area.')


    def icrn(self,ic,u):
        return u/ic

    def calc_icrn(self):
        
        popt, pcov = curve_fit(self.icrn,self.i0_corrected,self.rn)
        self.fitted_icrn=popt[0]
        return[popt[0]]
    
        


    def plot_jj_icrn(self,save_plot):
        '''Plots the normal resistance over the critical current'''
        pltrange=np.linspace(min(self.i0_corrected),max(self.i0_corrected),1000)
        fig, ax=plt.subplots()
        ax.annotate('$I_cR_{N,Fit}=$'+str(round(self.fitted_icrn)) + 'mV'+'\n $I_cR_{N,calc}=$' + str(round(self.j_c_corr*self.rho_n,0))+'mV',xy=(.9,.75),xycoords='figure fraction',horizontalalignment='right',verticalalignment='top')
        plt.plot(self.i0_corrected,self.rn,'.',label='corrected data')
        plt.plot(pltrange,self.icrn(pltrange,self.fitted_icrn),label='fitted $I_cR_N$ curve')
        plt.xlabel('$I_{c}$ [$\mu A$]')
        plt.ylabel('$R_N [\Omega]$')
        plt.legend()
        if save_plot == True:
            today = str(date.today())
            plt.savefig(today+'-jj-Ic_Rn_linear.png')
        plt.show()
        print()



        

    def plot_jj_icrn_loglog(self,save_plot):
        '''Plots the normal resistance over the critical current'''
        pltrange=np.linspace(min(self.i0_corrected),max(self.i0_corrected),1000)
        fig, ax=plt.subplots()
        ax.annotate('$I_cR_{N,Fit}=$'+str(round(self.fitted_icrn)) + 'mV'+'\n $I_cR_{N,calc}=$' + str(round(self.j_c_corr*self.rho_n,0))+'mV',xy=(.9,.75),xycoords='figure fraction',horizontalalignment='right',verticalalignment='top')
        plt.loglog(self.i0_corrected,self.rn,'.',label='corrected data')
        plt.loglog(pltrange,self.icrn(pltrange,self.fitted_icrn),label='fitted $I_cR_N$ curve')
        plt.xlabel('$I_{c}$ [$\mu A$]')
        plt.ylabel('$R_N [\Omega]$')
        plt.legend()
        if save_plot == True:
            today = str(date.today())
            plt.savefig(today+'-jj-Ic_Rn_loglog.png')        
        plt.show()
        print()






    def plot_jj_crit_current(self,save_plot):
        fig, ax = plt.subplots()
        plt.plot(1/(self.r300),self.i0_corrected,'.')
        plt.xlabel('1/R(300K) [$\Omega^{-1}$]')
        plt.ylabel('$I_{0}$ [$\mu A $]')
        #plt.legend()
        if save_plot == True:
            today = str(date.today())
            plt.savefig(today+'-jj-inverse-r300-crit-current.png')
        plt.show()

    def plot_jj_pointwise(self,point_index,save_plot):
        if point_index >= len(self.r300):
            print('Point index out of bounds!')
            point_index = point_index % len(self.r300)
            print ('Plot instead point_index = '+str(point_index))
        fig, ax = plt.subplots(2,1)
        ax[0].plot(1/(self.r300),self.i0_corrected,'.')
        ax[0].plot(1/(self.r300[point_index]),self.i0_corrected[point_index],'o')
        ax[0].set_xlabel('1/r(300K) [$\Omega^{-1}$]')
        ax[0].set_ylabel('$I_{0}$ [$\mu A $]')
        ax[1].plot(1/(self.r300*self.area_corrected),self.crit_current_density,'.')
        ax[1].plot(1/(self.r300[point_index]*self.area_corrected[point_index]),
                    self.crit_current_density[point_index],'o')
        ax[1].set_xlabel('1/rho(300K) [$\Omega^{-1} \mu m^{-2}$]')
        ax[1].set_ylabel('$J_{c}$ [$\mu A / \mu m^{2}$]')
        if save_plot == True:
            today = str(date.today())
            plt.savefig(today+'-point'+str(point_index)+'.png')
        fig.tight_layout()
        plt.show()
        
    def select_diameter(self,diameter,save_plot):
        if diameter not in self.diameter_design:
            print('For JJ size = '+str(diameter)+' \u03BCm')
            print('No JJ with this size in data set!')
            return 0
        r300_diameter = np.array([])
        i0_corrected_diameter = np.array([])
        area_corrected_diameter = np.array([])
        crit_current_density_diameter = np.array([])
        for i in range(len(self.r300)):
            if diameter == self.diameter_design[i]:
                r300_diameter = np.append(r300_diameter,self.r300[i])
                i0_corrected_diameter = np.append(i0_corrected_diameter,self.i0_corrected[i])
                area_corrected_diameter = np.append(area_corrected_diameter,self.area_corrected[i])
                crit_current_density_diameter = np.append(crit_current_density_diameter,
                                                          self.crit_current_density[i])
        fig, ax = plt.subplots(2,1)
        ax[0].plot(1/(r300_diameter),i0_corrected_diameter,'.')
        #ax[0].plot(1/(self.r300[point_index]),self.i0_corrected[point_index],'o')
        ax[0].set_xlabel('1/r(300K) [$\Omega^{-1}$]')
        ax[0].set_ylabel('$I_{0}$ [$\mu A $]')
        ax[1].plot(1/(r300_diameter*area_corrected_diameter),
                    crit_current_density_diameter,'.')
        #ax[1].plot(1/(self.r300[point_index]*self.area_corrected[point_index]),
        #            self.crit_current_density[point_index],'o')
        ax[1].set_xlabel('1/rho(300K) [$\Omega^{-1} \mu m^{-2}$]')
        ax[1].set_ylabel('$J_{c}$ [$\mu A / \mu m^{2}$]')
        if save_plot == True:
            today = str(date.today())
            plt.savefig(today+'-jjsize'+str(diameter)+'.png')
        fig.tight_layout()
        print('JJ size = '+str(diameter)+' \u03BCm')
        plt.show()

    def plot_jj_pointwise_all(self,save_plot):
        for i in range(len(self.r300)):
            print('Point No. '+str(i)+' highlighted')
            self.plot_jj_pointwise(i,save_plot)

    def plot_jj_pointwise_select_diameter_all(self,diameter,save_plot):
        if diameter not in self.diameter_design:
            print('For JJ size = '+str(diameter)+' \u03BCm')
            print('No JJ with this size in data set!')
            return 0
        r300_diameter = np.array([])
        i0_corrected_diameter = np.array([])
        area_corrected_diameter = np.array([])
        crit_current_density_diameter = np.array([])
        for i in range(len(self.r300)):
            if diameter == self.diameter_design[i]:
                r300_diameter = np.append(r300_diameter,self.r300[i])
                i0_corrected_diameter = np.append(i0_corrected_diameter,self.i0_corrected[i])
                area_corrected_diameter = np.append(area_corrected_diameter,self.area_corrected[i])
                crit_current_density_diameter = np.append(crit_current_density_diameter,
                                                          self.crit_current_density[i])
        for i in range(len(r300_diameter)):
            fig, ax = plt.subplots(2,1)
            ax[0].plot(1/(r300_diameter),i0_corrected_diameter,'.')
            ax[0].plot(1/(r300_diameter[i]),i0_corrected_diameter[i],'o')
            ax[0].set_xlabel('1/r(300K) [$\Omega^{-1}$]')
            ax[0].set_ylabel('$I_{0}$ [$\mu A $]')
            ax[1].plot(1/(r300_diameter*area_corrected_diameter),
                        crit_current_density_diameter,'.')
            ax[1].plot(1/(r300_diameter[i]*area_corrected_diameter[i]),
                        crit_current_density_diameter[i],'o')
            ax[1].set_xlabel('1/rho(300K) [$\Omega^{-1} \mu m^{-2}$]')
            ax[1].set_ylabel('$J_{c}$ [$\mu A / \mu m^{2}$]')
            if save_plot == True:
                today = str(date.today())
                plt.savefig(today+'-point'+str(i)+'-jjsize'+str(diameter)+'.png')
                fig.tight_layout()
                print('JJ size = '+str(diameter)+' \u03BCm, point no. '+str(i)+' highlighted')
                plt.show()

            

""" Main programm """

# testrun
"""
# test data input (2019-07-10)
path = '/home/private/fherr/gvfs/smb-share:domain=KIP,server=twix2,share=bolo,user=bolo/data/JosephsonJunctions/HDSQ12/HDSQ12w2/Chip_1A16'
file = 'Results_SingleJunctions_1A16_noanod.dat'
test = DataInput(path,file)
"""

# controll parameters
plot_data = True
plot_data_pointwise = False
save = True
startvalue = 3
#folder='T:/data/JosephsonJunctions/HDSQ15/HDSQ15w1'
names=['1A7']



r300 = np.array([])
i0pos= np.array([])
i0neg = np.array([])
size = np.array([])
rnpos = np.array([])
rnneg = np.array([])
print('Read data')
for name in names:
    print(name)
    #data = DataInput(folder+'/Chip_'+name,'Results_SingleJunctions_'+name+'.txt')
    data = DataInput(folder,name+'_data.txt')
    r300 = np.append(r300,data.r300())
    i0pos = np.append(i0pos,data.i0pos())
    i0neg = np.append(i0neg,data.i0neg())
    size = np.append(size,data.size())
    rnpos = np.append(rnpos,data.rnpos())
    rnneg = np.append(rnneg,data.rnneg())

test = CorrectJJData(r300,i0pos,i0neg,size,rnpos,rnneg)
test.jj_area_correction()
test.crit_current_suppression_correction(startvalue)
test.calc_jj_crit_current_density()
test.calc_icrn()
if plot_data == True:
    test.plot_jj_area_rn(plot_design = True,save_plot = save)
    # test.plot_jj_area_resistance(plot_design = True,save_plot = save)
    test.plot_jj_corr_area_i0(save_plot = save)
    test.plot_jj_crit_current_density(save_plot = save)
    test.plot_jj_crit_current(save_plot = save)
    test.plot_jj_icrn(save_plot = save)
    test.plot_jj_icrn_loglog(save_plot = save)
    if plot_data_pointwise == True:
        test.plot_jj_pointwise_all(save_plot = save)
        test.select_diameter(diameter = 4,save_plot = save)
        test.select_diameter(diameter = 4.5,save_plot = save)
        test.select_diameter(diameter = 5,save_plot = save)
        test.select_diameter(diameter = 6,save_plot = save)
        test.select_diameter(diameter = 8,save_plot = save)
        test.plot_jj_pointwise_select_diameter_all(diameter = 4,save_plot = save)
        test.plot_jj_pointwise_select_diameter_all(diameter = 4.5,save_plot = save)
        test.plot_jj_pointwise_select_diameter_all(diameter = 5,save_plot = save)
        test.plot_jj_pointwise_select_diameter_all(diameter = 6,save_plot = save)
        test.plot_jj_pointwise_select_diameter_all(diameter = 8,save_plot = save)



Read data
1A7


FileNotFoundError: [Errno 2] File b'T:\\data\\JosephsonJunctions\\HDSQ15\\HDSQ15w1\\Chip_1A7/Chip_1A7/Results_SingleJunctions_1A7.txt' does not exist: b'T:\\data\\JosephsonJunctions\\HDSQ15\\HDSQ15w1\\Chip_1A7/Chip_1A7/Results_SingleJunctions_1A7.txt'

In [2]:
import os 
os.path.dirname(os.path.realpath("jjanalysis-nls"))
folder=os.path.dirname(os.path.realpath("jjanalysis-nls"))
print(folder)

T:\data\JosephsonJunctions\HDSQ15\HDSQ15w1\Chip_1A7
